In [1]:
from dotenv import load_dotenv
import os

In [4]:
import xml.etree.ElementTree as ET
# Load variables from .env file into the environment
load_dotenv()

def search_in_list(list, keyword):
    for item in list:
        if keyword in item:
            return item
    return False

def convert():
    print("convert")
    # 1. Read the Draw.io file
    with open(os.getenv("SRC_FILE"), "r") as file:
        drawio_data = file.read()

    # 2. Extract the XML data
    start_tag = "<diagram"
    end_tag = "</diagram>"
    start_index = drawio_data.find(start_tag)
    end_index = drawio_data.find(end_tag) + len(end_tag)
    xml_data = drawio_data[start_index:end_index]
    # print(xml_data)
    print(os.getenv("DEST_XML_FILE"))
    if os.getenv("DEST_XML_FILE"):
        with open(os.getenv("DEST_XML_FILE"), "w") as file: file.writelines(xml_data)
    
    # 3. Process the XML data
    root = ET.fromstring(xml_data)

    blocks = {}
    inputs = {}
    edges = {}
    edges_msg = {}
    with open(os.getenv("DEST_FILE"), "w") as file:


        for mx_cell in root.findall(".//mxCell"):
            vertex = mx_cell.get("vertex", '')
            edge = mx_cell.get("edge", '')
            id = mx_cell.get("id", '')
            parent = mx_cell.get("parent", '')
            source = mx_cell.get("source", '')
            target = mx_cell.get("target", '')
            value = mx_cell.get("value", '').replace(" ", " ")
            styles = mx_cell.get("style", '').split(";")

            
            data = {}
            shape = search_in_list(styles, "shape")
            if shape != False:
                shape = shape.split("=")[1].replace("parallelogram", "input")
                data = {'mode':shape, 'previous':'', 'id':id, 'value':value, 'msg':''}

                if shape == 'input': inputs[id] = data
                blocks[id] = data
            msg = search_in_list(styles, "edgeLabel")
            if msg != False:
                data = {'parent':parent, 'id':id, 'value':value}
                edges_msg[id] = data
            
            if edge == '1':
                data = {'id':id, 'source':source, 'target':target, 'value':''}
                edges[id] = data
            
            
        
        # fuse edges with their messages
        for msg in edges_msg:
            target_edge_id = edges_msg[msg]['parent']
            edge_msg = edges_msg[msg]['value']
            edges[target_edge_id]['value'] = edge_msg
        

        # fuse inputs with their edge msg
        for edge in edges:
            target_input = edges[edge]['target']
            msg = edges[edge]['value']
            blocks[target_input]['msg'] = msg
            blocks[target_input]['previous'] = edges[edge]['source']

        # find the starting block
        starting_block_id = None
        for block in blocks:
            if starting_block_id == None:
                this_block = blocks[block]
                if this_block['previous'] == '': starting_block_id = this_block['id']

        # follow the tail from the starting block
        block_flow = []
        block_flow.append(blocks[starting_block_id])
        for i in blocks:
            for block in blocks:
                last_block_flow = block_flow[len(block_flow)-1]
                this_block = blocks[block]
                if this_block['previous'] == last_block_flow['id']:
                    block_flow.append(this_block)
            
        # print(block_flow)
        # display
        for block in block_flow:
            if block['mode'] == 'input':
                file.writelines(block['value'])
                file.writelines(" = ")
                file.writelines("input('"+block['msg']+"')")
                file.writelines("\n")

convert()

convert
/home/axlireza/Projects/diagrams-integration/integrate-flowchart-to-python/demo_flowchart.xml


In [ ]:
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Define the function to be executed when a change occurs
def on_file_change():
    print("File changed!")

# Define the event handler class
class FileChangeHandler(FileSystemEventHandler):
    def on_modified(self, event):
        # Call the function when a modification event occurs
        convert()

# Create an observer and attach the event handler
observer = Observer()
observer.schedule(FileChangeHandler(), path=os.getenv("SRC_FILE"), recursive=False)

# Start the observer
observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    # Stop the observer if interrupted
    observer.stop()

# Wait until the observer thread completes its execution
observer.join()